<a href="https://colab.research.google.com/github/mohripan/Kaggle-Training/blob/main/Bangsat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [46]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

--2021-07-03 13:23:55--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322873683 (308M) [application/zip]
Saving to: ‘/tmp/rockpaperscissors.zip’

/tmp/rockpapersciss 100%[===================>] 307.92M  5.78MB/s    in 51s     

2021-07-03 13:24:47 (6.05 MB/s) - ‘/tmp/rockpaperscissors.zip’ saved [322873683/322873683]



In [47]:
import zipfile, os

In [48]:
loc_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(loc_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/rockpaperscissors/rps-cv-images'

In [49]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Conv2D, MaxPooling2D,Activation,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [50]:
train_datagen = ImageDataGenerator( rescale = 1.0/255,
                                          rotation_range=40,
                                          width_shift_range=0.25,
                                          height_shift_range=0.25,
                                          shear_range=0.25,
                                          zoom_range=0.25,
                                          horizontal_flip=True,
                                          fill_mode='nearest',
                                           validation_split=0.4
                                  )


train_generator = train_datagen.flow_from_directory(base_dir,
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    target_size=(80, 60),
                                                   subset='training')

validation_datagen = ImageDataGenerator(rescale = 1.0/255,
                                        validation_split=0.4)


validation_generator = validation_datagen.flow_from_directory(base_dir,
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    target_size=(80, 60),
                                                    subset='validation')

Found 1314 images belonging to 3 classes.
Found 874 images belonging to 3 classes.


In [51]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(80, 60, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

model.summary()

model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 78, 58, 32)        896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 39, 29, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 37, 27, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 18, 13, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 7488)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               1917184   
_________________________________________________________________
dense_11 (Dense)             (None, 128)              

In [52]:
#from tensorflow.keras.callbacks import ReduceLROnPlateau
#learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
#                                            patience=2,
#                                            verbose=1,
#                                            factor=0.5,
#                                            min_lr=0.000001)

In [53]:
early_stopping = EarlyStopping(monitor='val_loss',min_delta = 0, patience = 8, verbose = 1, restore_best_weights=True)

In [54]:
filepath= "final_model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max', save_weights_only=False)

In [55]:
callbacks = [
        checkpoint,
        early_stopping
    ]

In [56]:
his=model.fit(train_generator,validation_data=validation_generator, epochs = 100,
                    verbose = 1, callbacks=callbacks)

Epoch 1/100
42/42 [==============================] - 7s 149ms/step - loss: 1.0885 - accuracy: 0.3813 - val_loss: 1.0573 - val_accuracy: 0.4485

Epoch 00001: val_accuracy improved from -inf to 0.44851, saving model to final_model.h5
Epoch 2/100
42/42 [==============================] - 6s 142ms/step - loss: 1.0228 - accuracy: 0.4954 - val_loss: 0.7827 - val_accuracy: 0.8124

Epoch 00002: val_accuracy improved from 0.44851 to 0.81236, saving model to final_model.h5
Epoch 3/100
42/42 [==============================] - 6s 140ms/step - loss: 0.9785 - accuracy: 0.5327 - val_loss: 0.5973 - val_accuracy: 0.8444

Epoch 00003: val_accuracy improved from 0.81236 to 0.84439, saving model to final_model.h5
Epoch 4/100
42/42 [==============================] - 6s 141ms/step - loss: 0.7541 - accuracy: 0.6743 - val_loss: 0.4636 - val_accuracy: 0.7952

Epoch 00004: val_accuracy did not improve from 0.84439
Epoch 5/100
42/42 [==============================] - 6s 139ms/step - loss: 0.7133 - accuracy: 0.679

In [57]:
his.history['accuracy']

[0.38127854466438293,
 0.4954337775707245,
 0.5327244997024536,
 0.6742770075798035,
 0.6796042323112488,
 0.7275494933128357,
 0.7579908967018127,
 0.7739726305007935,
 0.7503805160522461,
 0.77625572681427,
 0.8561643958091736,
 0.8637747168540955,
 0.8843227028846741,
 0.9063926935195923,
 0.9048706293106079,
 0.8858447670936584,
 0.9254185557365417,
 0.922374427318573,
 0.9200913310050964,
 0.9269406199455261,
 0.922374427318573,
 0.9117199182510376,
 0.9208523631095886,
 0.9337899684906006,
 0.9254185557365417]